In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd


# 베이스 경로 설정
BASE_DIR = '/content/drive/MyDrive/Dacon_FakeText/'

Mounted at /content/drive


In [ ]:
import numpy as np
import torch
import torch.nn as nn
import os
from tqdm import tqdm


In [ ]:
# Test 데이터 임베딩 로드
X_test = np.load(BASE_DIR + 'data/embeddings/test_concat.npy')
print(f"✅ Test 임베딩 Shape: {X_test.shape}")

# Test 메타 정보 로드 (ID 필요)
test_df = pd.read_csv(BASE_DIR + 'data/test.csv')
test_ids = test_df['ID'].values

✅ Test 임베딩 Shape: (1962, 775)


In [ ]:
# MLP 모델 클래스 (학습 때 사용한 구조와 완전 동일해야 함)
class MLP(nn.Module):
    def __init__(self, input_dim):
        super(MLP, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.layers(x)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
n_folds = 5
fold_preds = []

print("\nFold별 Test 예측 시작...")


✅ Fold별 Test 예측 시작...


In [ ]:
# Fold별 모델 불러와서 예측
for fold in range(1, n_folds + 1):
    print(f"\n=== Fold {fold} 모델 예측 중 ===")

    model = MLP(input_dim=X_test.shape[1]).to(device)
    model_path = BASE_DIR + f'model/mlp_fold{fold}.pt'
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()

    preds = []
    with torch.no_grad():
        for i in range(0, len(X_test), 1024):  # Test 배치사이즈 조정 가능 (ex: 512~2048)
            batch_x = torch.tensor(X_test[i:i+1024], dtype=torch.float32).to(device)
            batch_pred = model(batch_x).squeeze().cpu().numpy()
            preds.extend(batch_pred)

    fold_preds.append(np.array(preds))
    print(f"Fold {fold} 예측 완료! Shape: {len(preds)}")



=== Fold 1 모델 예측 중 ===
✅ Fold 1 예측 완료! Shape: 1962

=== Fold 2 모델 예측 중 ===
✅ Fold 2 예측 완료! Shape: 1962

=== Fold 3 모델 예측 중 ===
✅ Fold 3 예측 완료! Shape: 1962

=== Fold 4 모델 예측 중 ===
✅ Fold 4 예측 완료! Shape: 1962

=== Fold 5 모델 예측 중 ===
✅ Fold 5 예측 완료! Shape: 1962


In [ ]:
# Fold별 예측값 평균 (앙상블)
final_preds = np.mean(fold_preds, axis=0)
print(f"\n최종 앙상블 예측 완료! Shape: {final_preds.shape}")

# 제출파일 생성
submission = pd.DataFrame({
    'ID': test_ids,
    'generated': final_preds
})

output_path = BASE_DIR + '/output/sbert_concat_mlp_5fold.csv'
submission.to_csv(output_path, index=False, encoding='utf-8')
print(f"\n제출파일 저장 완료: {output_path}")


✅ 최종 앙상블 예측 완료! Shape: (1962,)

✅ 제출파일 저장 완료: /content/drive/MyDrive/Dacon_FakeText//output/sbert_concat_mlp_5fold.csv
